In [1]:
import numpy as np

# Hyperparams
N = 10

# I think d is the death rate. vary from 0.05 to 4
# field measurements are typically 0.1 - 0.3
d = 0

# self-limitation parameter. fixed at 1 in simulations
# changes the density by 1 / s i.e to get real answer divide by whatever s is
s = 1
alpha = 1

# Define a random adjacency matrix
# important, Aij != Aji
A = np.random.randint(0, 2, size=(N, N))

# Can't use normal here b/c negative. paper says 'a prob distribution' but not which one
# mean value is gamma (from data) and width is delta (not sure yet what that is)
# delta < gamma
# I think use a uniform dist
# okay I finanlly got it, uniform dist with mean gamma and 'width' (std)? delta
# eqn 23 in SI section 3 details the prob dist
# if delta is 0 then all weights are gamma so, 
gamma = np.random.uniform(0, 1, size=(N, N))

Ag = A * gamma
# integrate eqn 1 via 4th order runge-kutta algo until system reaches fixed point use t = 0.01, steady state at around 1e5
# init weights uniformly (0, 1] i guess
# similar to the process in SI section III with P(gamma_i_j)




In [3]:
from RungeKutta import RK4

def f(i, t, x, y):
    outer_sum = 0
    for j1 in range(N):

        inner_sum = 0
        for j2 in range(N):
            inner_sum += A[i][j] * x[j2]

        outer_sum += A[i][j1] * gamma[i][j1] * x[j1] / (alpha + inner_sum)
    return - d/s + 1/s + outer_sum

# Create system of equations
xs = []
for i in range(N):
    xs.append(lambda t, x, y: f(i, t, x, y))

print()